In [1]:
# добавления пути до модуля spin_nn
import sys
import os

project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_path not in sys.path:
    sys.path.append(project_path)

In [2]:
from spin_nn.model import MSKModel
from spin_nn.utils import load_mnist
from spin_nn.training import train
from spin_nn.visualization import *

import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

In [3]:
model = MSKModel.load_weights("/Users/rknzhl/projects/spin_glass/code/data/20241219_173925_epochs/weights_epoch_100.json")

Weights and structure loaded from /Users/rknzhl/projects/spin_glass/code/data/20241219_173925_epochs/weights_epoch_100.json


In [ ]:
def get_iteraction_matrix(weights):
    total_size = sum(w.shape[0] for w in weights) + weights[-1].shape[1]
    interaction_matrix = np.zeros((total_size, total_size))

    offset = 0
    for i, weight in enumerate(weights):
        rows, cols = weight.shape
        interaction_matrix[offset:offset+rows, offset:offset+cols] = weight
        interaction_matrix[offset:offset+cols, offset:offset+rows] = weight.T
        offset += rows

In [ ]:
J = get_iteraction_matrix(model.weights)

In [4]:

def build_M_matrix(J, beta):
    N = J.shape[0]
    # Вычисляем a_i = sum_{ell} J_{iell}^2 для каждой строки
    a = np.sum(J**2, axis=1)  # вектор длины N
    # Формируем M
    # M = beta * J - beta^2 * diag(a)
    M = beta * J - np.diag(beta**2 * a)
    return M

def find_min_eigval(J, beta):
    N = J.shape[0]
    M = build_M_matrix(J, beta)
    A = np.eye(N) - M  # I_N - M
    eigvals = np.linalg.eigvalsh(A)  # для симметричной матрицы
    return np.min(eigvals)

In [ ]:
betas = np.linspace(0, 5, 100)  # допустим, ищем в диапазоне от 0 до 5
min_eigs = []
for b in betas:
    val = find_min_eigval(J, b)
    min_eigs.append(val)

# Ищем бету, при которой min_eig максимально близко к 0
# Аргумент минимального по абсолютному значению собственноого числа
idx_closest = np.argmin(np.abs(min_eigs))
beta_crit = betas[idx_closest]
print("Approximate critical beta:", beta_crit)

# Строим график минимального собственного числа от беты
plt.figure()
plt.plot(betas, min_eigs, marker='o')
plt.axhline(0, color='r', linestyle='--')
plt.xlabel(r'$\beta$')
plt.ylabel('Minimum eigenvalue of $(I - M)$')
plt.title('Minimum eigenvalue vs beta')
plt.grid(True)
plt.show()